Pattern -> Descriptors -> Embeddings -> Coordinates (by patt index)

  -->  MIDI Drum Assignment -> Notes In Channels -> Flattening Type Results (by patt index) ==> for all flattening types.
  
---
Build and test NN for each (flat, coord) combo & report MAE results.

In [1]:
## Imports ##
import os
import numpy as np
import sys
import random
import importlib

## TODO: double check these need to be imported in this file
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as tdata
import torch.backends.mps
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
## File Locations (GitHub repo dir) ##
dir=os.getcwd()
os.listdir(dir)

import descriptors as desc
import functions as func
import flatten as flat
import neuralnetwork

/Users/peterclark/opt/anaconda3/envs/main/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Using cpu device


In [3]:
## File Locations (Google Drive) ##
#drive.mount('/content/drive/MyDrive/Tapping_Force_Experiment_Data')

'''
Mounting my drive doesn't work by code, but does by side panel.
We will need to figure this

dir = os.chdir('/content/drive/MyDrive/Tapping_Force_Experiment_Data')
os.listdir(dir)
import descriptors as desc
import functions as func
'''

"\nMounting my drive doesn't work by code, but does by side panel.\nWe will need to figure this\n\ndir = os.chdir('/content/drive/MyDrive/Tapping_Force_Experiment_Data')\nos.listdir(dir)\nimport descriptors as desc\nimport functions as func\n"

In [4]:
## Patterns ##
all_pattlists, all_names = func.rootfolder2pattlists("midi/","allinstruments")


In [ ]:
## Descriptors & Embedding Coordinates ~23s ##
d = desc.lopl2descriptors(all_pattlists)

mds_pos = func.d_mtx_2_mds(d)

In [33]:
## Convert Patterns (Full MIDI --> (8?) 3-Channel) ##
patterns = [] # 3 channel patterns
importlib.reload(flat)

for patt in all_pattlists:
    p = flat.get_LMH(patt)
    patterns.append(p)
print(patterns[1])

[[1], [], [2], [], [1], [], [2], [], [1], [], [2], [], [1], [], [2], []]


### Analytical Flattening Types:

Types: | Combo: | FuncRef: | Index: |
|---|:---:|---|---:|
**Onset Density** | (only) | [0,0,0] | 0 |
&emsp; &emsp; +| Metrical Strength  | [0,1,0] | 1 |
&emsp; &emsp; +| Syncopation  | [0,0,1] | 2 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [0,1,1] | 3 |
--- | --- | --- | ---
**Frequency Weighted <br>Onset Density** | (only) | [1,0,0] | 4 |
&emsp; &emsp; +| Metrical Strength  | [1,1,0] | 5 |
&emsp; &emsp; +| Syncopation  | [1,0,1] | 6 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [1,1,1] | 7 |
--- | --- | --- | ---
**Pattern Relative <br>Onset Density** | (only) | [2,0,0] | 8 |
&emsp; &emsp; +| Metrical Strength  | [2,1,0] | 9 |
&emsp; &emsp; +| Syncopation  | [2,0,1] | 10 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [2,1,1] | 11 |
--- | --- | --- | ---
**Note Presence** | (only/mono) | [3,0,0] | 12 |
&emsp; &emsp; +| Metrical Strength  | [3,1,0] | 13 |
&emsp; &emsp; +| Syncopation  | [3,0,1] | 14 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [3,1,1] | 15 |
&emsp; &emsp; +| Polyphonic Syncopation  | [3,0,2] | 16 |




In [22]:
# density types
onset_density=0
freq_weighted_onset_density=1
relative_onset_density=2
note_presence=3
# meter
no_meter=0
meter=1
# sync
no_sync=0
sync=1
polysync=2

In [34]:
importlib.reload(flat)
## Convert 3-channel to all 17 flat types. ##

n_patterns = len(patterns)
types = 17
flat_names = ['OnsetDen', 'OnsetDen_Meter', 'OnsetDen_Sync','OnsetDen_Sync_Meter',
              'FreqWeightOnsetDen', 'FreqWeightOnsetDen_Meter', 'FreqWeightOnsetDen_Sync','FreqWeightOnsetDen_Sync_Meter',
              'RelOnsetDen', 'RelOnsetDen_Meter', 'RelOnsetDen_Sync','RelOnsetDen_Sync_Meter',
              'Mono', 'Mono_Meter', 'Mono_Sync','Mono_Sync_Meter', 'Mono_PolySync']
len_patt = 16

# test
# patterns=patterns[13:20]

flatterns = np.array([[[0.0 for z in range(len_patt)] for y in range(types)] for x in range(n_patterns)], dtype=float) 
for p in range(len(patterns)):
    pattern = patterns[p]
# Onset Density
    flatterns[p][0] = flat.flat(pattern, onset_density, no_meter, no_sync)
    flatterns[p][1] = flat.flat(pattern, onset_density, meter, no_sync)
    flatterns[p][2] = flat.flat(pattern, onset_density, no_meter, sync)
    flatterns[p][3] = flat.flat(pattern, onset_density, meter, sync)
# Frequency Weighted Onset Density
    flatterns[p][4] = flat.flat(pattern, freq_weighted_onset_density, no_meter, no_sync)
    flatterns[p][5] = flat.flat(pattern, freq_weighted_onset_density, meter, no_sync)
    flatterns[p][6] = flat.flat(pattern, freq_weighted_onset_density, no_meter, sync)
    flatterns[p][7] = flat.flat(pattern, freq_weighted_onset_density, meter, sync)
# Pattern Relatiive Onset Density
    flatterns[p][8] = flat.flat(pattern, relative_onset_density, no_meter, no_sync)
    flatterns[p][9] = flat.flat(pattern, relative_onset_density, meter, no_sync)
    flatterns[p][10] = flat.flat(pattern, relative_onset_density, no_meter, sync)
    flatterns[p][11] = flat.flat(pattern, relative_onset_density, meter, sync)
# Note Presence
    flatterns[p][12] = flat.flat(pattern, note_presence, no_meter, no_sync)
    flatterns[p][13] = flat.flat(pattern, note_presence, meter, no_sync)
    flatterns[p][14] = flat.flat(pattern, note_presence, no_meter, sync)
    flatterns[p][15] = flat.flat(pattern, note_presence, meter, sync)
    flatterns[p][16] = flat.flat(pattern, note_presence, no_meter, polysync)
    if p %100==0:
        print(f"[x] - {p}/{len(patterns)}")
print(f"[x] - {p+1}/{len(patterns)}")

[x] - 0/1378
[x] - 100/1378
[x] - 200/1378
[x] - 300/1378
[x] - 400/1378
[x] - 500/1378
[x] - 600/1378
[x] - 700/1378
[x] - 800/1378
[x] - 900/1378
[x] - 1000/1378
[x] - 1100/1378
[x] - 1200/1378
[x] - 1300/1378
[x] - 1378/1378


In [35]:
''' CHANGE THIS TO DESIRED FLATTENING TYPE'''
flat_type = 16

""" print(flatterns.shape)
flattern = flatterns[:,flat_type,:]
print(flat_names[flat_type])
print(np.round(flattern[13:19],4)) """

for i in range(flat_type):
    print(flatterns.shape)
    flattern = flatterns[:,i,:]
    print(flat_names[i])
    print(flattern[824])


# TODO:
# Add 10 fold cross evaluation in NN build pipeline
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
# Return results and move print to this file



(1378, 17, 16)
OnsetDen
[[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]]
(1378, 17, 16)
OnsetDen_Meter
[[1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]
 [1.  0.  0.4 0.  0.6 0.  0.4 0.  0.8 0.  0.4 0.  0.6 0.  0.4 0. ]]


In [ ]:
## Load? ##
_load = False
model_dir=''

## Define model structure ##
model_layers=4
len_pattern=16 # input layer
firstlayer=32
secondlayer=32
thirdlayer=16
fourthlayer=16
coords=2 # output layer

## Build / Load model ##
if _load:
    model=neuralnetwork.load_model(model_dir)
else:
    model = neuralnetwork.build_model(len_pattern=16,firstlayer=32,secondlayer=32,thirdlayer=16,fourthlayer=16,coords=2)

## Select model hyperparameters ##
learning_rate = 0.001
batch_size = 32
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 200


train_loss = 0.0
test_loss = 0.0

In [ ]:
seed=np.random.randint(0,1000) ## for traintest splits. 
folds=10

kfold = KFold(n_splits=folds, shuffle=True)


In [ ]:
## For all Flat: NN build + (train + test)-10fold ## WARNING: LONGTIME!